In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/processed_df.csv')

In [3]:
df = df.reset_index(drop=True)


In [4]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
df

,comment_text,toxic,harassment,obscene,threat,insult,identity_hate
0,please please you little jewish,1,0,1,0,1,1
1,as drew often out on the say the fact that you...,0,0,0,0,0,0
2,so which will you promise not to reference in ...,0,0,0,0,0,0
3,off this is none of your concern its between m...,1,1,1,0,1,0
4,smaller unsigned comment added by,0,0,0,0,0,0
...,...,...,...,...,...,...,...
15682,ho hum the republican in full effect,0,0,0,0,0,0
15683,in your mother ass and pussy i want suck your ...,1,1,1,0,1,0
15684,you think than or romanian,0,0,0,0,0,0
15685,his mother is a slutty whore,1,0,1,0,1,1


In [6]:
labels = ['toxic', 'insult', 'threat', 'identity_hate', 'harassment', 'obscene']
print(df[labels].sum())

toxic            4208
insult           2648
threat           1958
identity_hate    2661
harassment       1519
obscene          2831
dtype: int64


In [7]:
labels = ['toxic', 'insult', 'threat', 'identity_hate', 'harassment', 'obscene']
negative_counts = (df[labels] == 0).sum()
print("Count of negative (0) for each label:")
print(negative_counts)

Count of negative (0) for each label:
toxic            11479
insult           13039
threat           13729
identity_hate    13026
harassment       14168
obscene          12856
dtype: int64


In [8]:
X = df["comment_text"]
y = df[["toxic", "harassment", "obscene", "threat", "insult", "identity_hate"]]

In [ ]:
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_recall_curve
from transformers.modeling_outputs import SequenceClassifierOutput

# === Prepare data ===
X_list = X.tolist() if hasattr(X, 'tolist') else X
y_array = y.values if hasattr(y, 'values') else np.array(y)

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
encodings = tokenizer(X_list, truncation=True, padding=True, max_length=128)

class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item
    def __len__(self):
        return len(self.labels)

dataset = ToxicDataset(encodings, y_array)

train_idx, val_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)

# === Calculate pos_weight for imbalanced data ===
pos_counts = y_array.sum(axis=0)
neg_counts = len(y_array) - pos_counts
pos_weights = neg_counts / (pos_counts + 1e-5)
pos_weights = torch.tensor(pos_weights, dtype=torch.float)

# === Custom model with pos_weight ===
class WeightedDistilBertForSequenceClassification(DistilBertForSequenceClassification):
    def __init__(self, config, pos_weight=None):
        super().__init__(config)
        self.pos_weight = pos_weight

    def forward(self, input_ids=None, attention_mask=None, head_mask=None, inputs_embeds=None, labels=None, **kwargs):
        outputs = self.distilbert(
            input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
        hidden_state = outputs[0]
        pooled_output = hidden_state[:, 0]
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=self.pos_weight.to(logits.device))
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states if self.config.output_hidden_states else None,
            attentions=outputs.attentions if self.config.output_attentions else None,
        )

model = WeightedDistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=y_array.shape[1],
    problem_type="multi_label_classification",
    pos_weight=pos_weights
)

# === Compute metrics ===
optimal_thresholds_cache = {}

def compute_optimal_thresholds(logits, labels):
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    thresholds = []
    for i in range(labels.shape[1]):
        precision, recall, thresh = precision_recall_curve(labels[:, i], probs[:, i])
        f1_scores = 2 * precision * recall / (precision + recall + 1e-8)
        best_thresh = thresh[np.argmax(f1_scores)] if len(thresh) > 0 else 0.5
        thresholds.append(best_thresh)
    return np.array(thresholds)

def compute_metrics(pred):
    logits, labels = pred.predictions, pred.label_ids
    probs = torch.sigmoid(torch.tensor(logits)).numpy()

    global optimal_thresholds_cache
    if not optimal_thresholds_cache:
        optimal_thresholds_cache = compute_optimal_thresholds(logits, labels)

    preds = (probs >= optimal_thresholds_cache).astype(int)

    f1_micro = f1_score(labels, preds, average='micro')
    f1_macro = f1_score(labels, preds, average='macro')

    return {"micro_f1": f1_micro, "macro_f1": f1_macro}

# === Training Arguments ===
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    logging_dir='./logs',
    fp16=True,
    gradient_accumulation_steps=2,
    save_total_limit=2,
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# === Train ===
trainer.train()

# === Evaluate after training ===
eval_results = trainer.evaluate()
print("Eval results:", eval_results)


Some weights of WeightedDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.968800
1000,0.581700
1500,0.432700
2000,0.323800
2500,0.243700
3000,0.194800
3500,0.156000
4000,0.123800
4500,0.100900
5000,0.085300


Eval results: {'eval_loss': 0.879681408405304, 'eval_micro_f1': 0.8035951082952704, 'eval_macro_f1': 0.8015938750980256, 'eval_runtime': 3.2308, 'eval_samples_per_second': 971.278, 'eval_steps_per_second': 60.976, 'epoch': 14.963057324840765}


In [48]:
!pip install onnx


In [ ]:
import torch

model.eval()
model.to('cpu')
sample_text = X.iloc[0] if hasattr(X, 'iloc') else X[0]
dummy_input = tokenizer(sample_text, return_tensors="pt")

# Export to ONNX
torch.onnx.export(
    model,
    (dummy_input['input_ids'], dummy_input['attention_mask']),
    "distilbert_multilabel__final2.onnx",
    input_names=['input_ids', 'attention_mask'],
    output_names=['output'],
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_length'},
        'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
        'output': {0: 'batch_size'}
    },
    opset_version=17 
)



In [42]:
!pip install onnxruntime


In [ ]:
import onnxruntime as ort
import numpy as np

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
ort_session = ort.InferenceSession("distilbert_multilabel.onnx")

def predict_onnx(texts):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="np")
    inputs = {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask']
    }
    outputs = ort_session.run(None, inputs)
    logits = outputs[0]
    probs = 1 / (1 + np.exp(-logits)) 
    return probs

texts = [
    
    "You're really annoying sometimes.",

    
    "I can't wait to see you fail miserably.",

   
    "This is a damn stupid idea!",

   
    "If you don't stop, I will make you regret it.",


    "You're such a worthless idiot.",

    
    "People like you shouldn't be allowed here.",

    
    "Thank you for your help!",

    
    "You're a disgusting failure and a joke.",

    
    "I’m going to find you and make your life miserable.",

    
    "What the hell is wrong with you?",

    
    "",

    "You have a great sense of humor."
]

probs = predict_onnx(texts)
print(probs)  

[[2.65553109e-02 3.38864326e-03 1.33970019e-03 3.67361940e-02
  4.38689749e-04 1.50397664e-03]
 [1.39771905e-02 2.56213779e-03 9.63928818e-04 5.44698583e-03
  3.92415049e-03 5.19679300e-03]
 [9.97954011e-01 1.33037835e-01 8.87043953e-01 6.47376478e-03
  1.46127045e-02 1.48628885e-03]
 [7.60420272e-03 1.89105712e-03 4.48465953e-03 3.12681615e-01
  2.40549285e-04 1.09803316e-03]
 [9.93564904e-01 8.93474877e-01 4.75253493e-01 2.75896792e-03
  9.85886931e-01 8.03654664e-04]
 [3.79441112e-01 3.91122361e-04 8.90205440e-04 1.13191744e-02
  1.86984278e-02 4.39175181e-02]
 [1.14328344e-03 2.37872475e-03 2.69458559e-03 2.06959923e-03
  1.40145677e-03 4.00811853e-03]
 [9.94031250e-01 1.49226010e-01 4.57586981e-02 5.37187001e-03
  6.42541111e-01 5.70316217e-04]
 [9.73599136e-01 2.59919818e-02 3.31315920e-02 6.95503533e-01
  2.58135229e-01 2.55350536e-03]
 [9.18976843e-01 1.44401728e-03 3.03773135e-01 4.53162082e-02
  6.22231164e-04 7.45758601e-03]
 [1.27456635e-01 7.32969679e-03 8.54126550e-03 1.9

In [ ]:
import numpy as np


thresholds = {
    "toxic": 0.50,
    "harassment": 0.50,
    "obscene": 0.50,
    "threat": 0.50,
    "insult": 0.50,
    "identity_hate": 0.50
}

label_names = ["toxic", "harassment", "obscene", "threat", "insult", "identity_hate"]

def apply_thresholds(probs, thresholds, label_names):
    preds = np.zeros_like(probs, dtype=int)
    for i, label in enumerate(label_names):
        thresh = thresholds[label]
        preds[:, i] = (probs[:, i] >= thresh).astype(int)
    return preds


preds = apply_thresholds(probs, thresholds, label_names)
print(preds)


[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 1 0 0 0]
 [0 0 0 0 0 0]
 [1 1 0 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 1 0]
 [1 0 0 1 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [53]:
label_names = ["toxic", "harassment", "obscene", "threat", "insult", "identity_hate"]
for i, row in enumerate(preds):
    labels = [label_names[j] for j, val in enumerate(row) if val == 1]
    print(f"Sample {i + 1}: {labels if labels else 'No toxic labels'}")

Sample 1: No toxic labels
Sample 2: No toxic labels
Sample 3: ['toxic', 'obscene']
Sample 4: No toxic labels
Sample 5: ['toxic', 'harassment', 'insult']
Sample 6: No toxic labels
Sample 7: No toxic labels
Sample 8: ['toxic', 'insult']
Sample 9: ['toxic', 'threat']
Sample 10: ['toxic']
Sample 11: No toxic labels
Sample 12: No toxic labels


In [ ]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
tokenizer.save_pretrained("./distilbert-tokenizer")


('./distilbert-tokenizer/tokenizer_config.json',
 './distilbert-tokenizer/special_tokens_map.json',
 './distilbert-tokenizer/vocab.txt',
 './distilbert-tokenizer/added_tokens.json',
 './distilbert-tokenizer/tokenizer.json')

In [55]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
tokenizer.save_pretrained("./distilbert-tokenizer")

('./distilbert-tokenizer/tokenizer_config.json',
 './distilbert-tokenizer/special_tokens_map.json',
 './distilbert-tokenizer/vocab.txt',
 './distilbert-tokenizer/added_tokens.json',
 './distilbert-tokenizer/tokenizer.json')

In [ ]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)

zip_folder('./distilbert-tokenizer', 'distilbert-tokenizer.zip')
